## 預測價格

In [62]:
import os
import math
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from xgboost import XGBRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score
from joblib import Parallel, delayed

# --------------------------
# 1. 讀取資料
# --------------------------
df = pd.read_csv('/Users/yuchingchen/Documents/專題/cleaned_data/short_flight.csv')

# --------------------------
# 2. 欄位定義
# --------------------------
cat_cols  = ['出發時段','出發機場代號','抵達時段','航空聯盟','機型分類','假期','是否為平日']
group_col = '抵達機場代號'
num_cols  = ['飛行時間_分鐘','經濟指標','機場指標','competing_flights']
log_col   = '平均價格_log'
true_col  = '平均價格'           # 直接用原始的平均價格欄
pred_col  = '預測_平均價格'

# --------------------------
# 3. 分艙等並標記真實價格
# --------------------------
eco_raw = df[df['艙等']=='經濟艙'][cat_cols + [group_col] + num_cols + [log_col] + [true_col]].copy()
biz_raw = df[df['艙等']=='商務艙'][cat_cols + [group_col] + num_cols + [log_col] + [true_col]].copy()

# --------------------------
# 4. 編碼 & 標準化
# --------------------------
def encode_and_scale(df_raw):
    df_enc = pd.get_dummies(df_raw, columns=cat_cols, drop_first=True)
    scaler = StandardScaler()
    df_enc[num_cols] = scaler.fit_transform(df_enc[num_cols])
    return df_enc, scaler

eco_enc, eco_scaler = encode_and_scale(eco_raw)
biz_enc, biz_scaler = encode_and_scale(biz_raw)

# --------------------------
# 5. 對齊特徵欄位
# --------------------------
common_feats = list(set(eco_enc.columns) & set(biz_enc.columns))
for col in [log_col, true_col, group_col]:
    if col in common_feats:
        common_feats.remove(col)

# --------------------------
# 6. 訓練模型 & 效能評估 & 半方差校正
# --------------------------
seed = 123

eco_model = XGBRegressor(n_estimators=100, random_state=seed, n_jobs=-1)
eco_model.fit(eco_enc[common_feats], eco_enc[log_col])
eco_log_pred = eco_model.predict(eco_enc[common_feats])
mse_log_eco = mean_squared_error(eco_enc[log_col], eco_log_pred)
r2_log_eco  = r2_score(eco_enc[log_col], eco_log_pred)
print(f"[經濟艙][log-space] MSE = {mse_log_eco:.4f}, R2 = {r2_log_eco:.4f}")
sigma2_eco = np.mean((eco_enc[log_col] - eco_log_pred)**2)

biz_model = RandomForestRegressor(n_estimators=100, random_state=seed, n_jobs=-1)
biz_model.fit(biz_enc[common_feats], biz_enc[log_col])
biz_log_pred = biz_model.predict(biz_enc[common_feats])
mse_log_biz = mean_squared_error(biz_enc[log_col], biz_log_pred)
r2_log_biz  = r2_score(biz_enc[log_col], biz_log_pred)
print(f"[商務艙][log-space] MSE = {mse_log_biz:.4f}, R2 = {r2_log_biz:.4f}")
sigma2_biz = np.mean((biz_enc[log_col] - biz_log_pred)**2)

# --------------------------
# 7. 真實組合 & 並行化預測函式
# --------------------------
def get_unique_combinations(df_raw, code):
    sub = df_raw[df_raw[group_col]==code]
    return sub[cat_cols + num_cols + [true_col]].drop_duplicates().reset_index(drop=True)

def predict_and_save(df_raw, model, scaler, sigma2, prefix, n_jobs=-1, batch_size=500):
    os.makedirs('predict_data', exist_ok=True)
    for code in sorted(df_raw[group_col].unique()):
        combos = get_unique_combinations(df_raw, code)

        # 編碼 & 補齊
        enc = pd.get_dummies(combos, columns=cat_cols, drop_first=True)
        enc = enc.reindex(columns=common_feats, fill_value=0)

        # 標準化
        enc[num_cols] = scaler.transform(combos[num_cols])

        # 並行預測 log
        log_preds = np.concatenate(
            Parallel(n_jobs=n_jobs)(
                delayed(model.predict)(enc.iloc[i:i+batch_size])
                for i in range(0, len(enc), batch_size)
            )
        )

        # bias‐corrected back‐transform
        combos[pred_col] = np.expm1(log_preds + 0.5 * sigma2)
        combos[group_col]  = code

        # 去重
        combos = combos.drop_duplicates(subset=cat_cols + num_cols)

        # 存檔
        fn = f'predict_data/short/{prefix}_{code}.csv'
        combos.to_csv(fn, index=False)
        print(f"✅ 已存：{fn} （共 {len(combos)} 筆）")

# --------------------------
# 8. 執行預測並存檔
# --------------------------
predict_and_save(eco_raw, eco_model, eco_scaler, sigma2_eco, 'eco')
predict_and_save(biz_raw, biz_model, biz_scaler, sigma2_biz, 'biz')

[經濟艙][log-space] MSE = 0.0342, R2 = 0.9052
[商務艙][log-space] MSE = 0.0049, R2 = 0.9427
✅ 已存：predict_data/short/eco_BKK.csv （共 226 筆）
✅ 已存：predict_data/short/eco_GMP.csv （共 39 筆）
✅ 已存：predict_data/short/eco_HKG.csv （共 782 筆）
✅ 已存：predict_data/short/eco_HND.csv （共 71 筆）
✅ 已存：predict_data/short/eco_ICN.csv （共 193 筆）
✅ 已存：predict_data/short/eco_NRT.csv （共 303 筆）
✅ 已存：predict_data/short/eco_SIN.csv （共 122 筆）
✅ 已存：predict_data/short/biz_BKK.csv （共 119 筆）
✅ 已存：predict_data/short/biz_GMP.csv （共 17 筆）
✅ 已存：predict_data/short/biz_HKG.csv （共 296 筆）
✅ 已存：predict_data/short/biz_HND.csv （共 52 筆）
✅ 已存：predict_data/short/biz_ICN.csv （共 103 筆）
✅ 已存：predict_data/short/biz_NRT.csv （共 175 筆）
✅ 已存：predict_data/short/biz_SIN.csv （共 69 筆）
